# C++学习笔记

本文主要介绍C++中跟类有关的知识点。

## 第十章 对象和类

这一章主要是简单的介绍了C++中类的相关概念，主要的知识点有以下几点：

1. 在类中定义的方法都是默认内联的，这些方法也可以显式地内联定义。同样也可以不采用内联定义。如果使用内联定义一定要在头文件中，否则会引发错误。
2. 使用`const`关键字修饰的对象调用的方法也必须使用`const`关键字修饰，使用`const`关键字修饰的方法不能修改类的任何内容。
3. 在类的内部使用`this`关键字指代这个类对象的指针，`this`指针指向类对象本身，如果要返回类对象而非指针，那么应该使用`*`运算符`*this`。
4. 类中往往需要使用一些常量来规定例如数组的长度或者栈的大小，这种情况下可以使用作用域内的枚举或者静态`const`变量来实现。
5. C++11中可以使用`{}`来实例化对象，并不一定要显式调用构造方法。

### 内联的类方法

在C++中如果一个函数或方法声明为内联的，那么这个方法就需要在每个要使用它的程序文件中都定义一次，这种特性就决定了内联方法或者函数最好在头文件中被定义，因为每当头文件被`#include`的时候就会执行一次内联函数或方法的定义。

C++中在类中声明的方法默认就是内联的：

```C++
//person.h
class Person{
private:
    std::string name;
    int age;
    gender gen;
public:
    Person(){
        this->name = "unknown";
        this->age = 0;
        this->gen = male;
    }
    void show() const{
        cout << "The person " << this->name
             << "'s age is " << this->age
             << ", his/her gender is " << this->gen << endl;
    }
};
```
以上代码包含了`Person`类的构造方法和一个普通的成员方法，这两个方法都是以默认形式实现的，这说明这两个方法都是内联方法。这也就是为什么这个类能够在`person.h`这样的头文件中被定义而不用担心方法重复定义的问题，因为内联函数或方法本身就是要在每个使用的程序文件中都定义一次。另外，出了这种默认的写法，内敛方法也可以显示地去定义：

```C++
//person.h
class Person{
private:
    std::string name;
    int age;
    gender gen;
public:
    Person();
    void show() const;
};

inline Person::Person(){//inline关键字需要显式给出
    this->name = "unknown";
    this->age = 0;
    this->gen = male;
}

inline void Person::show() const{
    cout << "The person " << this->name
         << "'s age is " << this->age
         << ", his/her gender is " << this->gen << endl;
}
```

最后，类的成员方法还可以不以内联的方式来定义，这种定义方式就不能在头文件中了，因为这将会引发方法重复定义的错误：

```C++
//person.h
class Person{
private:
    std::string name;
    int age;
    gender gen;
public:
    Person();
    void show() const;
};

//person.cpp
void Person::show(){
    cout << "The person " << this->name
         << "'s age is " << this->age
         << ", his/her gender is " << this->gen << endl;
}
Person::Person(){
    this->name = "unknown";
    this->age = 0;
    this->gen = male;
}
```

### `const`关键字修饰的实例

使用`const`关键字修饰一个实例意味着这个实例是不可变的，这个不可变的实例调用的方法也应该是`const`关键字修饰的，否则就会印发错误：

```C++
//person.h
class Person{
private:
    std::string name;
    int age;
    gender gen;
public:
    Person(){
        this->name = "unknown";
        this->age = 0;
        this->gen = male;
    }
    void show() const{
        cout << "The person " << this->name
             << "'s age is " << this->age
             << ", his/her gender is " << this->gen << endl;
    }
};

//main.cpp
#include "Person.h"
using namespace std;
const Person p = Person();
//如果show()方法没有使用const关键字修饰，那么就会引发错误
p.show();
```

### 类中的常量

通常情况下，在类中如果需要声明一个常量，那么通常会想到的做法是声明一个不可变的成员变量：`cosnt int size = 15;`，但是这种做法往往是行不通的，因为这种类型的普通成员变量都是在类已经实例化之后赋值，如果要使用这种常量作为类中数组或者栈的大小，那么编译器就会引发错误，因为数组或者栈的大小需要在类实例化之前就要确定，然而这种形式下类实例化之前变量还没有被赋值。解决这种问题的方案有两个：

1. 类作用域内部的枚举。
2. 静态`const`变量。

使用类作用域内部的枚举是声明一个匿名枚举：

```C++
//person.h
class person{
private:
    enum egg{five=12};
    double array[five];
}
```
另一种方式是使用静态`const`变量，这种静态变量的生存周期于类无关，它是和整个程序的生存周期相同的，因此在类实例化之前，相应的变量就已经赋值了。但是需要说明是这里的变量一定要使用`const`和`static`两个关键字修饰：

```C++
//person.h
class person{
private:
    static const int size = 15;
    double array[size];
}
```

### 对象的初始化

C++11中既可以使用传统的调用构造方法的形式来实例化对象，也可以使用C++中新定义的方法来实例化：

```C++
//person.h
class Person{
private:
    std::string name;
    int age;
    gender gen;
public:
    Person(const std::string &name){
        this->name = name;
        this->age = 0;
        this->gen = male;
    }
    void show() const{
        cout << "The person " << this->name
             << "'s age is " << this->age
             << ", his/her gender is " << this->gen << endl;
    }
};

//main.cpp
const Person p = {"David"};
p.show();
```

这里虽然使用了`{}`的形式来初始化，但是在内部编译器还是根据传入的参数类型和个数（也就是特征标）来动态匹配调用相应的构造方法。

还有在命名空间中使用类和类的构造函数和析构函数相关知识比较简单，请看代码。

## 第十一章 使用类